In [27]:
## 简单的 exp to be ran
# todo 基于aug1的预训练权重初始化att
# todo 使用last模型全部测试后排序
# todo 训练p6模型
# todo 训练yolov6 v7作为baseline
# ongoing 训练1280下的全部改进版本

## 复杂的 idea to be implemented
# todo synth预训练 + nominal微调 + edge测试
# todo 基于bbox预测继续预测跑道角点位置 用于自动着陆 【不规则目标检测】

In [28]:
# from ultralytics import YOLO

# path_model = 'cfg/ultralytics/models/yolov8n-p2/yolov8n-p2_afpn.yaml'
# model = YOLO(path_model)
# results = model.train(
#     data='coco128.yaml',
#     epochs=1,
#     project="test",
#     plots=False)

In [29]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import scienceplots
plt.style.use('science')
plt.rcParams['text.usetex'] = False

# 获取结果文件
path = 'results/tables/metrics'
all_csv = os.listdir(path)
all_csv = [file for file in all_csv if file.endswith('csv')]

# 控制分辨率
use1280 = True
if use1280: 
    all_csv = [file for file in all_csv if (file.find('1280')!=-1) | (file.find('baseline')!=-1)]
    name_base_model = 'base_yolov8n_1280'
else:
    name_base_model = 'base_yolov8n'

def get_model_name(file):
    # 从文件名截取模型名称
    return file.split('metrics_')[1].split('.csv')[0]

# 使用map@5095或者map@.5
use_5095 = True

# 截取map@5095或者map@.5
map = []
for file in all_csv:
    df = pd.read_csv(os.path.join(path, file))
    if use_5095:
        _map = df.iloc[0:1, :].copy()
    else:
        _map = df.iloc[1:2, :].copy()
    _map['metrics'] = get_model_name(file)
    map.append(_map)

# 数据重构
map = pd.concat(map, axis=0).sort_values(by='test_real_nominal')
map.set_index('metrics', inplace=True)

def highlight_base_model_row(s):
    # 高亮name_base_model所在行
    return ['background-color: yellow' if s.name == name_base_model else '' for _ in s]

def highlight_cells(cell, reference_cell):
    # 如果cell的值大于reference_cell则高亮cell
    if cell > reference_cell:
        return 'background-color: red'
    return ''

def highlight_better(row):
    # 对row整行 某个cell大于reference_row的对应cell 则高亮
    reference_row = df.loc[name_base_model]
    return [highlight_cells(cell, reference_cell) for cell, reference_cell in zip(row, reference_row)]

# 应用这个样式到DataFrame
print('map@5095') if use_5095 else print('map@.5')
map.style.bar(
    align='left'
    ).apply(highlight_base_model_row, axis=1
    ).format("{:.3f}")

map@5095


,test_synth,test_real_nominal,test_real_edge
metrics,,,
baseline_ssd,0.674,0.508,0.261
baseline_yolov3,0.726,0.523,0.240
baseline_yolov5n,0.706,0.540,0.302
baseline_faster_rcnn,0.719,0.565,0.286
baseline_retinanet,0.659,0.593,0.331
baseline_centernet,0.801,0.613,0.218
base_yolov8n_1280,0.850,0.719,0.403
att_yolov8n_cpca_1280,0.848,0.721,0.389
att_yolov8n_eca_1280,0.842,0.722,0.418


In [30]:
# Baseline
df = map.copy()
df = df[df.index.str.contains('baseline_') | (df.index == name_base_model)]
df.style.apply(highlight_base_model_row, axis=1).apply(highlight_better, axis=1).format("{:.3f}")

,test_synth,test_real_nominal,test_real_edge
metrics,,,
baseline_ssd,0.674,0.508,0.261
baseline_yolov3,0.726,0.523,0.240
baseline_yolov5n,0.706,0.540,0.302
baseline_faster_rcnn,0.719,0.565,0.286
baseline_retinanet,0.659,0.593,0.331
baseline_centernet,0.801,0.613,0.218
base_yolov8n_1280,0.850,0.719,0.403


In [31]:
# Orin
df = map.copy()
df = df[df.index.str.contains('base_') | (df.index == name_base_model)]
df.style.apply(highlight_base_model_row, axis=1).apply(highlight_better, axis=1).format("{:.3f}")

,test_synth,test_real_nominal,test_real_edge
metrics,,,
base_yolov8n_1280,0.850,0.719,0.403
base_yolov8n-p2_1280,0.845,0.742,0.434


In [32]:
# Album
df = map.copy()
df = df[df.index.str.contains('aug_') | (df.index == name_base_model)]
df.style.apply(highlight_base_model_row, axis=1).apply(highlight_better, axis=1).format("{:.3f}")

,test_synth,test_real_nominal,test_real_edge
metrics,,,
base_yolov8n_1280,0.850,0.719,0.403
aug_yolov8n_aug5_1280,0.853,0.750,0.502


In [33]:
# Attention
df = map.copy()
df = df[df.index.str.contains('att_') | (df.index == name_base_model)]
df.style.apply(highlight_base_model_row, axis=1).apply(highlight_better, axis=1).format("{:.3f}")

,test_synth,test_real_nominal,test_real_edge
metrics,,,
base_yolov8n_1280,0.850,0.719,0.403
att_yolov8n_cpca_1280,0.848,0.721,0.389
att_yolov8n_eca_1280,0.842,0.722,0.418
att_yolov8n_se_1280,0.848,0.727,0.391
att_yolov8n_ema_1280,0.843,0.729,0.419
att_yolov8n_lska_1280,0.847,0.732,0.410
att_yolov8n_sa_1280,0.847,0.737,0.429
att_yolov8n_gam_1280,0.853,0.738,0.429
att_yolov8n_cbam_1280,0.850,0.741,0.428


In [34]:
# IoU
df = map.copy()
df = df[df.index.str.contains('iou_') | (df.index == name_base_model)]
df.style.apply(highlight_base_model_row, axis=1).apply(highlight_better, axis=1).format("{:.3f}")

,test_synth,test_real_nominal,test_real_edge
metrics,,,
base_yolov8n_1280,0.850,0.719,0.403


In [35]:
# Neck
df = map.copy()
df = df[df.index.str.contains('neck_') | (df.index == name_base_model)]
df.style.apply(highlight_base_model_row, axis=1).apply(highlight_better, axis=1).format("{:.3f}")

,test_synth,test_real_nominal,test_real_edge
metrics,,,
base_yolov8n_1280,0.850,0.719,0.403
neck_yolov8n_bifpn_1280,0.846,0.740,0.450
neck_yolov8n_afpn_1280,0.847,0.744,0.416


In [36]:
# Merge
df = map.copy()
df = df[df.index.str.contains('merge_') | (df.index == name_base_model)]
df.style.apply(highlight_base_model_row, axis=1).apply(highlight_better, axis=1).format("{:.3f}")

,test_synth,test_real_nominal,test_real_edge
metrics,,,
base_yolov8n_1280,0.850,0.719,0.403


In [37]:
# from src.utils.metric import ensemble_coco_metrics

# path_preds_json_list = [
#     # 'runs/ultralytics/merge/yolov8n_cpca_SIoU_aug1/test/test_real_edge/predictions_modified.json',
#     'runs/ultralytics/merge/yolov8n_lska_WIoU_aug1/test/test_real_edge/predictions_modified.json',
#     'runs/ultralytics/merge/yolov8n_ta_SIoU_aug1/test/test_real_edge/predictions_modified.json'
# ]
# path_instance_json = 'datasets/lard/annotations/instances_test_real_edge.json'
# all_metrics = ensemble_coco_metrics(path_preds_json_list, path_instance_json, verbose=False, ensemble_method='nmw')  # ['nms', 'soft', 'nmw', 'wbf']

In [38]:
# def highlight_last_col_max(row):
#     last_col = row.iloc[-1]  # 使用 iloc 获取最后一列的值
#     is_max = last_col > row.iloc[:-1].max()  # 比较最后一列的值与其他列的最大值
#     return ['background-color: red' if (col == last_col and is_max) else '' for col in row]
# all_metrics.style.apply(highlight_last_col_max, axis=1)